In [9]:
import os
import random
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchaudio
import torchaudio.transforms
from torch.utils.data import Dataset, DataLoader, Subset
from sklearn.model_selection import train_test_split
# Import các thư viện metrics mới
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    classification_report, confusion_matrix, roc_auc_score,
    roc_curve, auc
)
from torchvision.transforms import v2 as T
from tqdm.auto import tqdm
# Import Matplotlib để vẽ biểu đồ
import matplotlib.pyplot as plt

import warnings 
warnings.simplefilter(action='ignore', category=FutureWarning)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

Using device: cuda


In [2]:
class TimeShift(torch.nn.Module):
    """ Dịch chuyển spectrogram theo trục thời gian một cách ngẫu nhiên. """
    def __init__(self, p=0.5, max_shift_percent=0.4):
        super().__init__()
        self.p = p
        self.max_shift_percent = max_shift_percent

    def forward(self, spec):
        if torch.rand(1) < self.p:
            time_steps = spec.shape[-1]
            shift_amount = int(torch.rand(1) * self.max_shift_percent * time_steps)
            if torch.rand(1) < 0.5:
                shift_amount = -shift_amount
            return torch.roll(spec, shifts=shift_amount, dims=-1)
        return spec

class SpecNoise(torch.nn.Module):
    """ Thêm nhiễu Gaussian trực tiếp vào spectrogram. """
    def __init__(self, p=0.5, noise_level=0.05):
        super().__init__()
        self.p = p
        self.noise_level = noise_level

    def forward(self, spec):
        if torch.rand(1) < self.p:
            noise = torch.randn_like(spec) * self.noise_level
            return spec + noise
        return spec

train_transform = T.Compose([
    torchaudio.transforms.TimeMasking(time_mask_param=80),
    torchaudio.transforms.FrequencyMasking(freq_mask_param=25),
    TimeShift(p=0.5, max_shift_percent=0.4),
    SpecNoise(p=0.3, noise_level=0.03),
])

val_transform = None
print("Đã định nghĩa xong pipeline Augmentation mới cho Spectrogram.")

Đã định nghĩa xong pipeline Augmentation mới cho Spectrogram.


In [4]:
def mixup_data(x, y, alpha=0.4, use_cuda=True):
    if alpha > 0: lam = np.random.beta(alpha, alpha)
    else: lam = 1
    batch_size = x.size()[0]
    if use_cuda: index = torch.randperm(batch_size).cuda()
    else: index = torch.randperm(batch_size)
    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

def mixup_criterion(criterion, pred, y_a, y_b, lam):
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

In [5]:
class DynamicGraphAttention(nn.Module):
    def __init__(self, in_channels, num_nodes=32, heads=8, dropout=0.1):
        super().__init__()
        self.num_nodes, self.heads = num_nodes, heads
        self.head_dim = in_channels // heads
        self.dropout = nn.Dropout(dropout)
        self.query = nn.Linear(in_channels, in_channels)
        self.key = nn.Linear(in_channels, in_channels)
        self.value = nn.Linear(in_channels, in_channels)
        self.out = nn.Linear(in_channels, in_channels)
        self.scale = nn.Parameter(torch.tensor(self.head_dim**-0.5))
        self.pos_enc = nn.Parameter(torch.randn(1, num_nodes, in_channels))

    def forward(self, x):
        B, C, H, W = x.size()
        x_flat = x.flatten(2).transpose(1, 2)
        num_pixels = H * W
        N = min(self.num_nodes, num_pixels)
        _, indices = torch.topk(torch.norm(x_flat, dim=-1), N, dim=-1)
        indices_expanded = indices.unsqueeze(-1).expand(-1, -1, C)
        x_nodes = torch.gather(x_flat, 1, indices_expanded)
        if self.pos_enc.size(1) >= N:
            x_nodes = x_nodes + self.pos_enc[:, :N, :]
        q = self.query(x_nodes).view(B, N, self.heads, self.head_dim).transpose(1, 2)
        k = self.key(x_nodes).view(B, N, self.heads, self.head_dim).transpose(1, 2)
        v = self.value(x_nodes).view(B, N, self.heads, self.head_dim).transpose(1, 2)
        attn_weights = F.softmax(torch.matmul(q, k.transpose(-2, -1)) * self.scale, dim=-1)
        attn_output = torch.matmul(self.dropout(attn_weights), v).transpose(1, 2).contiguous().view(B, N, C)
        out_nodes = self.out(attn_output)
        out_flat = torch.zeros_like(x_flat)
        out_flat.scatter_(1, indices_expanded, out_nodes)
        out = out_flat.transpose(1, 2).view(B, C, H, W)
        return x + out

class TemporalAttentionAggregator(nn.Module):
    def __init__(self, in_channels, num_heads=4):
        super().__init__()
        self.attention_net = nn.MultiheadAttention(embed_dim=in_channels, num_heads=num_heads, batch_first=True)
        self.norm = nn.LayerNorm(in_channels)

    def forward(self, x):
        if x.dim() != 4: raise ValueError("Input must be 4D tensor")
        B, C, H, W = x.size()
        x_flat = x.flatten(2).transpose(1, 2)
        x_norm = self.norm(x_flat)
        attn_output, _ = self.attention_net(x_norm, x_norm, x_norm)
        return torch.mean(attn_output, dim=1)

class InvertedResidual(nn.Module):
    def __init__(self, in_channels, out_channels, stride, expansion=6):
        super().__init__()
        hidden_dim = in_channels * expansion
        self.use_residual = in_channels == out_channels and stride == 1
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, hidden_dim, 1, bias=False), nn.BatchNorm2d(hidden_dim), nn.SiLU(inplace=True),
            nn.Conv2d(hidden_dim, hidden_dim, 3, stride, 1, groups=hidden_dim, bias=False), nn.BatchNorm2d(hidden_dim), nn.SiLU(inplace=True),
            nn.Conv2d(hidden_dim, out_channels, 1, bias=False), nn.BatchNorm2d(out_channels)
        )
        self.se = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(out_channels, out_channels // 16, 1), nn.SiLU(),
            nn.Conv2d(out_channels // 16, out_channels, 1), nn.Sigmoid()
        )

    def forward(self, x):
        out = self.conv(x)
        out = out * self.se(out)
        return x + out if self.use_residual else out

In [16]:
class PestLarvaNet_Medium_Encoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.config = [[1, 24, 2, 1], [6, 40, 3, 2], [6, 64, 3, 2], [6, 120, 3, 2], [6, 160, 2, 1]]
        self.stem = nn.Sequential(
            nn.Conv2d(1, 24, 3, 2, 1, bias=False),
            nn.BatchNorm2d(24),
            nn.SiLU(inplace=True)
        )
        layers = []
        in_channels = 24
        for t, c, n, s in self.config:
            for i in range(n):
                stride = s if i == 0 else 1
                layers.append(InvertedResidual(in_channels, c, stride, expansion=t))
                in_channels = c
            if c == 40:
                layers.append(DynamicGraphAttention(in_channels, num_nodes=32, heads=8))
        self.body = nn.Sequential(*layers)
        self.head_conv = nn.Conv2d(160, 384, 1, bias=False)
        self.aggregator = TemporalAttentionAggregator(384, num_heads=8)
    
    def forward(self, x):
        # Model xử lý đầu vào (B, H, W) -> (B, 1, H, W)
        if x.dim() == 3:
            x = x.unsqueeze(1)
        x = self.stem(x)
        x = self.body(x)
        x = self.head_conv(x)
        x = self.aggregator(x)
        return x

class PestLarvaNet_Medium_Classifier(nn.Module):
    def __init__(self, num_classes, pretrained_weights_path=None):
        super().__init__()
        self.encoder = PestLarvaNet_Medium_Encoder()
        self.classifier = nn.Sequential(
            nn.Linear(384, 192),
            nn.SiLU(inplace=True),
            nn.Dropout(0.4),
            nn.Linear(192, num_classes)
        )
        if pretrained_weights_path:
            print(f"Loading pretrained weights from: {pretrained_weights_path}")
            checkpoint = torch.load(pretrained_weights_path, map_location=device, weights_only=True)
            # Adjust the checkpoint loading to handle encoder weights
            state_dict = {k.replace('encoder.', ''): v for k, v in checkpoint.items() if k.startswith('encoder.')}
            self.encoder.load_state_dict(state_dict, strict=False)

    def forward(self, x):
        x = self.encoder(x)
        return self.classifier(x)

print("Đã định nghĩa xong kiến trúc model.")

Đã định nghĩa xong kiến trúc model.


In [17]:
class SpectrogramDataset(Dataset):
    # Sử dụng 'melspec_dir' trỏ trực tiếp đến thư mục chứa file .pt
    def __init__(self, metadata_df, melspec_dir, transform=None, target_tdim=626):
        super().__init__()
        self.metadata = metadata_df
        self.melspec_dir = melspec_dir 
        self.transform = transform
        self.target_tdim = target_tdim
        self.class_names = sorted(self.metadata['label'].unique())
        self.class_to_id = {name: i for i, name in enumerate(self.class_names)}

    def __len__(self):
        return len(self.metadata)

    def __getitem__(self, idx):
        row = self.metadata.iloc[idx]
        relative_path = row['file_path'] 
        file_path = os.path.join(self.melspec_dir, relative_path)
        label = self.class_to_id[row['label']]
        try:
            spectrogram = torch.load(file_path).to(torch.float32) # (H, W)
            current_tdim = spectrogram.shape[-1]
            if current_tdim > self.target_tdim:
                spectrogram = spectrogram[..., :self.target_tdim]
            elif current_tdim < self.target_tdim:
                spectrogram = F.pad(spectrogram, (0, self.target_tdim - current_tdim))
            if self.transform:
                spectrogram = self.transform(spectrogram)
            return spectrogram, label
        except Exception as e:
            print(f"Error loading or processing file {file_path}: {e}")
            return None

def collate_fn_remove_none(batch):
    batch = [item for item in batch if item is not None]
    if not batch: 
        return torch.tensor([]), torch.tensor([]) 
    samples, labels = zip(*batch)
    return torch.stack(samples), torch.tensor(labels)

print("Đã cập nhật xong SpectrogramDataset.")

def collate_fn_remove_none(batch):
    """ Lọc ra các item bị lỗi (None) từ __getitem__ trước khi tạo batch. """
    batch = [item for item in batch if item is not None]
    if not batch: 
        # Trả về tensor rỗng nếu cả batch bị lỗi
        return torch.tensor([]), torch.tensor([]) 
    samples, labels = zip(*batch)
    # torch.stack sẽ tạo (B, H, W)
    return torch.stack(samples), torch.tensor(labels)

print("Đã cập nhật xong SpectrogramDataset.")

Đã cập nhật xong SpectrogramDataset.
Đã cập nhật xong SpectrogramDataset.


In [18]:
def plot_roc_curve(val_labels, val_probs, class_names, save_path):
    """
    Vẽ và lưu biểu đồ ROC Curve (One-vs-Rest) cho từng lớp.
    """
    n_classes = len(class_names)
    fpr = dict()
    tpr = dict()
    roc_auc = dict()

    # Chuyển đổi labels sang one-hot
    val_labels_one_hot = F.one_hot(torch.tensor(val_labels), num_classes=n_classes).numpy()

    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(val_labels_one_hot[:, i], val_probs[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    plt.figure(figsize=(10, 8))
    
    # Vẽ đường ROC cho từng lớp
    for i in range(n_classes):
        plt.plot(fpr[i], tpr[i], label=f'Class {class_names[i]} (AUC = {roc_auc[i]:.4f})')

    plt.plot([0, 1], [0, 1], 'k--', label='Chance (AUC = 0.50)')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'Multi-class (One-vs-Rest) ROC Curve')
    plt.legend(loc="lower right")
    plt.grid(True)
    plt.savefig(save_path)
    plt.close() # Đóng plot để giải phóng bộ nhớ

def train_and_evaluate(
    model, 
    train_loader, 
    val_loader, 
    epochs, 
    results_dir,      # <<< THAY ĐỔI: Nhận thư mục output
    run_base_name,    # <<< THAY ĐỔI: Nhận tên cơ sở (ví dụ: 'Model_A_data_25.0percent')
    class_names 
):
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
    criterion = nn.CrossEntropyLoss()
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', factor=0.5, patience=5, verbose=False)
    scaler = torch.cuda.amp.GradScaler(enabled=torch.cuda.is_available())
    
    best_f1_weighted = 0.0 
    best_metrics_dict = {} # <<< MỚI: Lưu trữ dict metrics tốt nhất
    
    # <<< MỚI: Tạo đường dẫn động
    metrics_csv_path = os.path.join(results_dir, f"metrics_history_{run_base_name}.csv")
    best_checkpoint_path = os.path.join(results_dir, f"Best_{run_base_name}.pth")
    report_path = os.path.join(results_dir, f"Best_{run_base_name}_report.txt")
    cm_path = os.path.join(results_dir, f"Best_{run_base_name}_confusion_matrix.csv")
    auc_plot_path = os.path.join(results_dir, f"Best_{run_base_name}_AUC_ROC.png")

    # <<< MỚI: Header CSV đầy đủ
    csv_header = [
        'Epoch', 'Accuracy', 
        'Precision_micro', 'Precision_macro', 'Precision_w',
        'Recall_micro', 'Recall_macro', 'Recall_w',
        'F1_micro', 'F1_macro', 'F1_w',
        'AUC_macro'
    ]
    if not os.path.exists(metrics_csv_path):
        pd.DataFrame(columns=csv_header).to_csv(metrics_csv_path, index=False)
        
    for epoch in range(epochs):
        model.train()
        loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}", leave=False)
        for inputs, labels in loop:
            if inputs.shape[0] == 0: continue
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad(set_to_none=True)
            with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):
                outputs = model(inputs)
                loss = criterion(outputs, labels)
            scaler.scale(loss).backward(); scaler.step(optimizer); scaler.update()
            
        model.eval()
        val_preds, val_labels, val_probs = [], [], []
        
        with torch.no_grad():
            for inputs, labels in val_loader:
                if inputs.shape[0] == 0: continue
                inputs, labels = inputs.to(device), labels.to(device)
                with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):
                    outputs = model(inputs)
                probs = F.softmax(outputs, dim=1) 
                _, predicted = torch.max(outputs.data, 1)
                val_preds.extend(predicted.cpu().numpy())
                val_labels.extend(labels.cpu().numpy())
                val_probs.extend(probs.cpu().numpy())
                
        if len(val_labels) > 0:
            val_labels_np = np.array(val_labels)
            val_preds_np = np.array(val_preds)
            val_probs_np = np.array(val_probs)
            
            # <<< MỚI: Tính toán TẤT CẢ các chỉ số
            metrics_data = {'Epoch': epoch + 1}
            metrics_data['Accuracy'] = accuracy_score(val_labels_np, val_preds_np)
            
            # Precision
            metrics_data['Precision_micro'] = precision_score(val_labels_np, val_preds_np, average='micro', zero_division=0)
            metrics_data['Precision_macro'] = precision_score(val_labels_np, val_preds_np, average='macro', zero_division=0)
            metrics_data['Precision_w'] = precision_score(val_labels_np, val_preds_np, average='weighted', zero_division=0)
            
            # Recall
            metrics_data['Recall_micro'] = recall_score(val_labels_np, val_preds_np, average='micro', zero_division=0)
            metrics_data['Recall_macro'] = recall_score(val_labels_np, val_preds_np, average='macro', zero_division=0)
            metrics_data['Recall_w'] = recall_score(val_labels_np, val_preds_np, average='weighted', zero_division=0)
            
            # F1-Score
            metrics_data['F1_micro'] = f1_score(val_labels_np, val_preds_np, average='micro', zero_division=0)
            metrics_data['F1_macro'] = f1_score(val_labels_np, val_preds_np, average='macro', zero_division=0)
            metrics_data['F1_w'] = f1_score(val_labels_np, val_preds_np, average='weighted', zero_division=0)
            
            # AUC
            try:
                metrics_data['AUC_macro'] = roc_auc_score(val_labels_np, val_probs_np, multi_class='ovr', average='macro')
            except ValueError:
                metrics_data['AUC_macro'] = 0.0

            # Ghi vào file CSV (ghi MỌI epoch)
            pd.DataFrame([metrics_data]).to_csv(metrics_csv_path, mode='a', header=False, index=False)

            # Tiêu chí chính để lưu model (vẫn dùng F1-weighted)
            current_f1_weighted = metrics_data['F1_w']
            if current_f1_weighted > best_f1_weighted:
                best_f1_weighted = current_f1_weighted
                best_metrics_dict = metrics_data # <<< MỚI: Lưu lại dict metrics tốt nhất
                
                print(f"\nEpoch {epoch+1}: New best F1-weighted: {best_f1_weighted:.4f}")
                print(f"  F1-Macro: {metrics_data['F1_macro']:.4f}, AUC-Macro: {metrics_data['AUC_macro']:.4f}, Accuracy: {metrics_data['Accuracy']:.4f}")
                print(f"Saving best model artifacts to {results_dir}")
                
                # Lưu Checkpoint
                torch.save(model.state_dict(), best_checkpoint_path)
                
                # Lưu Báo cáo Phân loại (Classification Report)
                report = classification_report(val_labels_np, val_preds_np, target_names=class_names, digits=4, zero_division=0)
                with open(report_path, 'w', encoding='utf-8') as f:
                    f.write(report)
                    f.write(f"\n--- Summary Metrics ---\n")
                    for key, val in best_metrics_dict.items():
                        if key != 'Epoch': f.write(f"{key}: {val:.4f}\n")
                    
                # Lưu Ma trận Nhầm lẫn (Confusion Matrix)
                cm_df = pd.DataFrame(confusion_matrix(val_labels_np, val_preds_np), index=class_names, columns=class_names)
                cm_df.to_csv(cm_path)
                
                # <<< MỚI: Vẽ và lưu biểu đồ AUC ROC
                plot_roc_curve(val_labels_np, val_probs_np, class_names, auc_plot_path)

            scheduler.step(current_f1_weighted) 
            
    print(f"Finished Training for this run. Best F1-weighted: {best_f1_weighted:.4f}")
    return best_metrics_dict # <<< THAY ĐỔI: Trả về dict đầy đủ

In [19]:
# --- CẤU HÌNH ĐƯỜNG DẪN ---
# 1. MELSPEC_DIR: Đường dẫn đến thư mục CHỨA các file .pt
MELSPEC_DIR = r'C:\Users\PC\Downloads\Melspectrogram_finetune_NEW\Finetune_MelSpec' 

# 2. METADATA_PATH: Đường dẫn ĐẦY ĐỦ đến file .csv
METADATA_PATH = r'C:\Users\PC\Downloads\Melspectrogram_finetune_NEW\metadata_Finetune_MelSpec.csv'

# 3. BASE_OUTPUT_DIR: Thư mục GỐC để lưu TẤT CẢ các thư mục kết quả
#    (Lấy thư mục cha của file metadata)
BASE_OUTPUT_DIR = os.path.dirname(METADATA_PATH)
print(f"Tất cả kết quả sẽ được lưu vào các thư mục con trong: {BASE_OUTPUT_DIR}")

# 4. (Giữ nguyên) Đường dẫn đến checkpoint PRETRAINED
CHECKPOINT_BASE = r'C:\Users\PC\PestLarvaNet\SSL\audiotwin_checkpoints_medium\Final checkpoint'

# --- (Code thực thi) ---
checkpoint_files = [
    'run_1_lp1e-04_ep1e-06_lb0.001_audiotwin_pestlarvanet_medium_epoch_100.pth',
    'run_2_lp5e-04_ep1e-06_lb0.003_audiotwin_pestlarvanet_medium_epoch_100.pth',
    'run_3_lp1e-03_ep1e-06_lb0.005_audiotwin_pestlarvanet_medium_epoch_100.pth',
    'run_4_lp5e-03_ep1e-06_lb0.007_audiotwin_pestlarvanet_medium_epoch_100.pth',
    'run_5_lp1e-02_ep1e-06_lb0.010_audiotwin_pestlarvanet_medium_epoch_100.pth'
]
DATA_PERCENTAGES = [0.25, 0.5, 1.0]
results = {} 

print("Preparing dataset...")
full_metadata = pd.read_csv(METADATA_PATH)
num_classes = len(full_metadata['label'].unique())
print(f"Number of classes: {num_classes}")

indices = np.arange(len(full_metadata))
labels = full_metadata['label'].values

# Chia train/test (80/20)
train_indices, test_indices, _, _ = train_test_split(
    indices, labels, test_size=0.2, random_state=SEED, stratify=labels
)
print(f"Data split: {len(train_indices)} train, {len(test_indices)} test.")

# Tạo Test loader
test_metadata = full_metadata.iloc[test_indices]
test_dataset = SpectrogramDataset(metadata_df=test_metadata, melspec_dir=MELSPEC_DIR, transform=val_transform)
test_loader = DataLoader(test_dataset, batch_size=32, num_workers=0, collate_fn=collate_fn_remove_none)

# Lấy danh sách tên lớp
class_names_list = test_dataset.class_names
print(f"Class names: {class_names_list}")

model_configs = {
    f"PestLarvaNet (AudioTwin Run {i+1})": {
        "model_class": PestLarvaNet_Medium_Classifier,
        "args": {"num_classes": num_classes, "pretrained_weights_path": os.path.join(CHECKPOINT_BASE, checkpoint_files[i])}
    } for i in range(len(checkpoint_files))
}

Tất cả kết quả sẽ được lưu vào các thư mục con trong: C:\Users\PC\Downloads\Melspectrogram_finetune_NEW
Preparing dataset...
Number of classes: 8
Data split: 11240 train, 2810 test.
Class names: ['Gryllus_bimaculatus', 'Gryllus_campestris', 'Halyomorpha', 'Melolontha_melolontha', 'Myzus', 'Nezara', 'Trialeurodes', 'Tuta']


In [20]:
for model_name, config in model_configs.items():
    print(f"\n{'='*20} Evaluating: {model_name} {'='*20}")
    model_results = []
    
    # <<< MỚI: Tạo thư mục cho mô hình này
    model_folder_name = model_name.replace(' ', '_').replace('(', '').replace(')', '')
    model_output_dir = os.path.join(BASE_OUTPUT_DIR, model_folder_name)
    os.makedirs(model_output_dir, exist_ok=True)
    print(f"Lưu kết quả của model này vào: {model_output_dir}")

    for percent in DATA_PERCENTAGES:
        print(f"--- Training with {percent*100:.1f}% of labeled data ---")

        if percent < 1.0:
            stratify_labels = full_metadata['label'].iloc[train_indices].values
            subset_train_indices, _ = train_test_split(
                train_indices, train_size=percent, random_state=SEED, stratify=stratify_labels
            )
        else:
            subset_train_indices = train_indices

        train_metadata_subset = full_metadata.iloc[subset_train_indices]
        train_dataset = SpectrogramDataset(metadata_df=train_metadata_subset, melspec_dir=MELSPEC_DIR, transform=train_transform)
        train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0, collate_fn=collate_fn_remove_none)
        
        # <<< MỚI: Tạo thư mục con cho % dữ liệu này
        percent_str = f"data_{percent*100:.1f}percent"
        run_output_dir = os.path.join(model_output_dir, percent_str)
        os.makedirs(run_output_dir, exist_ok=True)
        
        # <<< MỚI: Tạo tên cơ sở (base name) cho lần chạy
        run_base_name = f"{model_folder_name}_{percent_str}"
        
        model = config["model_class"](**config["args"]).to(device)
        
        # <<< MỚI: Lời gọi hàm train_and_evaluate đã thay đổi
        best_metrics = train_and_evaluate(
            model, 
            train_loader, 
            test_loader, 
            epochs=100, 
            results_dir=run_output_dir,    # Đường dẫn đến thư mục con (VD: .../data_25.0percent/)
            run_base_name=run_base_name,   # Tên cơ sở (VD: 'PestLarvaNet_Run_1_data_25.0percent')
            class_names=class_names_list 
        )
        
        model_results.append({'percent': percent, 'metrics': best_metrics})
        if best_metrics: # Kiểm tra xem best_metrics có rỗng không
            print(f"==> Results with {percent*100:.1f}% data: Best F1-Weighted = {best_metrics.get('F1_w', 0):.4f}")
        else:
            print(f"==> No valid results for {percent*100:.1f}% data.")

    results[model_name] = model_results

print("\n--- FINAL RESULTS (Summary) ---")
for model_name, result_list in results.items():
    print(f"\nModel: {model_name}")
    for res in result_list:
        percent_str = f"{res['percent']*100:.1f}% data"
        if res['metrics']:
            print(f"  {percent_str}: F1-W={res['metrics']['F1_w']:.4f} | F1-M={res['metrics']['F1_macro']:.4f} | AUC={res['metrics']['AUC_macro']:.4f} | Acc={res['metrics']['Accuracy']:.4f}")
        else:
            print(f"  {percent_str}: No valid metrics recorded.")


==================== Evaluating: PestLarvaNet (AudioTwin Run 1) ====================
Lưu kết quả của model này vào: C:\Users\PC\Downloads\Melspectrogram_finetune_NEW\PestLarvaNet_AudioTwin_Run_1
--- Training with 25.0% of labeled data ---
Loading pretrained weights from: C:\Users\PC\PestLarvaNet\SSL\audiotwin_checkpoints_medium\Final checkpoint\run_1_lp1e-04_ep1e-06_lb0.001_audiotwin_pestlarvanet_medium_epoch_100.pth


C:\Users\PC\anaconda3\envs\FallDeteNet\lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/100:   0%|          | 0/88 [00:00<?, ?it/s]


Epoch 1: New best F1-weighted: 0.4536
  F1-Macro: 0.3937, AUC-Macro: 0.0000, Accuracy: 0.5078
Saving best model artifacts to C:\Users\PC\Downloads\Melspectrogram_finetune_NEW\PestLarvaNet_AudioTwin_Run_1\data_25.0percent


Epoch 2/100:   0%|          | 0/88 [00:00<?, ?it/s]

...

Epoch 97/100:   0%|          | 0/352 [00:00<?, ?it/s]

Epoch 98/100:   0%|          | 0/352 [00:00<?, ?it/s]

Epoch 99/100:   0%|          | 0/352 [00:00<?, ?it/s]

Epoch 100/100:   0%|          | 0/352 [00:00<?, ?it/s]

Finished Training for this run. Best F1-weighted: 0.9218
==> Results with 100.0% data: Best F1-Weighted = 0.9218

--- FINAL RESULTS (Summary) ---

Model: PestLarvaNet (AudioTwin Run 1)
  25.0% data: F1-W=0.7847 | F1-M=0.7560 | AUC=0.0000 | Acc=0.7854
  50.0% data: F1-W=0.8532 | F1-M=0.8362 | AUC=0.0000 | Acc=0.8534
  100.0% data: F1-W=0.9200 | F1-M=0.9174 | AUC=0.0000 | Acc=0.9199

Model: PestLarvaNet (AudioTwin Run 2)
  25.0% data: F1-W=0.7731 | F1-M=0.7433 | AUC=0.0000 | Acc=0.7737
  50.0% data: F1-W=0.8456 | F1-M=0.8281 | AUC=0.0000 | Acc=0.8459
  100.0% data: F1-W=0.9208 | F1-M=0.9156 | AUC=0.0000 | Acc=0.9206

Model: PestLarvaNet (AudioTwin Run 3)
  25.0% data: F1-W=0.8001 | F1-M=0.7767 | AUC=0.0000 | Acc=0.8007
  50.0% data: F1-W=0.8601 | F1-M=0.8448 | AUC=0.0000 | Acc=0.8598
  100.0% data: F1-W=0.9189 | F1-M=0.9119 | AUC=0.0000 | Acc=0.9189

Model: PestLarvaNet (AudioTwin Run 4)
  25.0% data: F1-W=0.7745 | F1-M=0.7400 | AUC=0.0000 | Acc=0.7754
  50.0% data: F1-W=0.8567 | F1-M=0.